In [1]:
import os
import re
import requests
from bs4 import BeautifulSoup

In [2]:
accepted_base_url = 'https://steamcommunity.com/workshop/browse/?appid=440&browsesort=accepted&section=mtxitems&browsefilter=accepted'
not_accepted_base_url = 'https://steamcommunity.com/workshop/browse/?appid=440&browsesort=trend&section=mtxitems'


In [3]:
def get_soup(url: str):
    return BeautifulSoup(requests.get(url).content, 'html.parser')


In [5]:
def download_item_by_id(folder_name: str, id: str):
    with open(f'{folder_name}{os.sep}{id}.html', "wb") as item_file:
        item_file.write(requests.get(
            f"https://steamcommunity.com/sharedfiles/filedetails/?id={id}").content)


In [4]:
def download_items(folder_name: str, base_url: str):
    os.makedirs(folder_name, exist_ok=True)

    base_soup = get_soup(base_url)

    page_count = int(base_soup.select('.pagelink')[-1].text.replace(',', ''))

    paging_info = base_soup.select_one('.workshopBrowsePagingInfo').text
    item_count = int(
        re.search(r'of ([\d,]+) entries', paging_info).group(1).replace(',', ''))

    print(page_count, 'pages', item_count, 'items')

    item_count = 1

    for page_number in range(1, page_count+1):
        print('started page', page_number)

        page_soup = get_soup(base_url+f'&p={page_number}')
        page_item_ids = [e.find('a').attrs['data-publishedfileid']
                         for e in page_soup.select('.workshopItem')]

        for id in page_item_ids:
            if os.path.exists(f'{folder_name}/{id}.html'):
                continue

            print('downloading item number', item_count, 'with id:', id)
            download_item_by_id(folder_name,id)

            item_count = item_count + 1


In [ ]:
download_items('accepted',accepted_base_url)

In [ ]:
download_items('not_accepted',not_accepted_base_url)